In [2]:
import pandas as pd
import tszip

In [6]:
regions = [
    ("ORF1AB", 266, 21555),
    ("S", 21563, 25384),
    ("ORF3", 25393, 26220),
    ("E", 26245, 26472),
    ("M", 26523, 27191),
    ("ORF6", 27202, 27387),
    ("ORF7a", 27394, 27759),
    ("ORF7b", 27756, 27887),
    ("ORF8", 27894, 28259),
    ("N", 28274, 29533),
    ("ORF10", 29558, 29674)]

def get_genome_region(pos): 
    for label, start, end in regions:
        if start <= pos <= end:
            return label
    return "non-coding"

ORF1AB_regions = [
    ("NSP1", 266, 805),
    ("NSP2", 806, 2719),
    ("NSP3", 2720, 8554),
    ("NSP4", 8555, 10054),
    ("NSP5", 10055, 10972),
    ("NSP6", 10973, 11842),
    ("NSP7", 11843, 12091),
    ("NSP8", 12092, 12685),
    ("NSP9", 12686, 13024),
    ("NSP10", 13025, 13441),
    ("NSP12", 13441, 16236),  # frameshift happens here
    ("NSP13", 16237, 18039),
    ("NSP14", 18040, 19620),
    ("NSP15", 19621, 20658),
    ("NSP16", 20659, 21552)
]

def get_ORF1AB_region(pos):
    for label, start, end in ORF1AB_regions:
        if start <= pos <= end:
            return label
    return ""

spike_subregions = [
    ("S1 - NTD - signal peptide", 21563, 21601),
    ("S1 - NTD", 21563, 22475),
    ("S1 - RBD", 22517, 23185),
    ("S1/2 cleavage site", 22517, 23185),
    ("S1 - other", 21563, 23617),
    ("S2 - Fusion peptide", 23924, 23980),
    ("S2 - HR1", 24296, 24514),
    ("HR2", 25049, 25201),
    ("TM domain", 25202, 25273),
    ("CTD", 25274, 25383),
    ("S2 - other", 23618, 25383)]

def get_spike_domain(pos):
    for label, start, end in spike_subregions:
        if start <= pos <= end:
            return label
    return ""

In [7]:
ts = tszip.load("sc2ts_viridian_v1.1.trees.tsz")


In [8]:
ts = tszip.load("sc2ts_viridian_v1.1.trees.tsz")
ARG_deletion_events = pd.read_csv('sc2ts_v1_2023-02-21_pr_pp_mp_aph_bps_pango_dated_deletion_events.csv', index_col=0)

ARG_deletion_events["region"] = ARG_deletion_events["start"].apply(get_genome_region)
ARG_deletion_events["ORF1AB_region"] = ARG_deletion_events["start"].apply(get_ORF1AB_region)
ARG_deletion_events["spike_region"] = ARG_deletion_events["start"].apply(get_spike_domain)
ARG_deletion_events["pango"] = ARG_deletion_events["node"].apply(lambda x: ts.node(x).metadata["pango"])
ARG_deletion_events["deletion"] = ARG_deletion_events.apply(lambda row: tuple(range(row["start"], row["start"] + row["length"])), axis=1)
ARG_deletion_events = ARG_deletion_events.sort_values(by = "min_inheritors", ascending = False)

In [9]:
ARG_deletion_events[ARG_deletion_events["max_inheritors"] > 10000].reset_index()

,index,start,node,length,max_inheritors,min_inheritors,region,ORF1AB_region,spike_region,pango,deletion
0,30086,28271,1436808,1,1118363,1118363,non-coding,,,B.1,"(28271,)"
1,27869,28248,220186,6,1114669,1114326,ORF8,,,B.1.617.2,"(28248, 28249, 28250, 28251, 28252, 28253)"
2,14712,22029,200039,6,1112410,1111939,S,,S1 - NTD,B.1.617.2,"(22029, 22030, 22031, 22032, 22033, 22034)"
3,8248,11288,1436802,4,871782,871762,ORF1AB,NSP6,,B.1.1.529,"(11288, 11289, 11290, 11291)"
4,8249,28362,1436802,9,871774,850272,N,,,B.1.1.529,"(28362, 28363, 28364, 28365, 28366, 28367, 283..."
5,8255,21633,822854,9,532072,531990,S,,S1 - NTD,BA.2,"(21633, 21634, 21635, 21636, 21637, 21638, 216..."
6,8254,11292,822854,5,531894,531867,ORF1AB,NSP6,,BA.2,"(11292, 11293, 11294, 11295, 11296)"
7,7952,6513,851246,3,340765,340751,ORF1AB,NSP3,,BA.1,"(6513, 6514, 6515)"
8,7953,11283,851246,5,339867,339865,ORF1AB,NSP6,,BA.1,"(11283, 11284, 11285, 11286, 11287)"
9,7954,21765,851246,6,339681,339472,S,,S1 - NTD,BA.1,"(21765, 21766, 21767, 21768, 21769, 21770)"


In [12]:
ARG_deletion_events[ARG_deletion_events["max_inheritors"] > 10000].reset_index().to_latex()

'\\begin{tabular}{lrrrrrrlllll}\n\\toprule\n & index & start & node & length & max_inheritors & min_inheritors & region & ORF1AB_region & spike_region & pango & deletion \\\\\n\\midrule\n0 & 30086 & 28271 & 1436808 & 1 & 1118363 & 1118363 & non-coding &  &  & B.1 & (28271,) \\\\\n1 & 27869 & 28248 & 220186 & 6 & 1114669 & 1114326 & ORF8 &  &  & B.1.617.2 & (28248, 28249, 28250, 28251, 28252, 28253) \\\\\n2 & 14712 & 22029 & 200039 & 6 & 1112410 & 1111939 & S &  & S1 - NTD & B.1.617.2 & (22029, 22030, 22031, 22032, 22033, 22034) \\\\\n3 & 8248 & 11288 & 1436802 & 4 & 871782 & 871762 & ORF1AB & NSP6 &  & B.1.1.529 & (11288, 11289, 11290, 11291) \\\\\n4 & 8249 & 28362 & 1436802 & 9 & 871774 & 850272 & N &  &  & B.1.1.529 & (28362, 28363, 28364, 28365, 28366, 28367, 28368, 28369, 28370) \\\\\n5 & 8255 & 21633 & 822854 & 9 & 532072 & 531990 & S &  & S1 - NTD & BA.2 & (21633, 21634, 21635, 21636, 21637, 21638, 21639, 21640, 21641) \\\\\n6 & 8254 & 11292 & 822854 & 5 & 531894 & 531867 & ORF1A

Table 1: Deletion events which have more than 10k inheritors. Of these 17 deletions, eight are located in the N-terminal domain of Spike, a previously noted site for key deletions. Many of these nodes represent the origins of major lineages, for example the 851246 node which represents the origin of BA.1, and the 86456 node which represents the origin of Alpha (see table 2). 

In [10]:
ARG_deletion_events[ARG_deletion_events["node"] == 86456]

,start,node,length,max_inheritors,min_inheritors,region,ORF1AB_region,spike_region,pango,deletion
8218,28271,86456,1,290446,290446,non-coding,,,B.1.1.7,"(28271,)"
8215,11288,86456,9,290408,290402,ORF1AB,NSP6,,B.1.1.7,"(11288, 11289, 11290, 11291, 11292, 11293, 112..."
8217,21991,86456,3,288831,288192,S,,S1 - NTD,B.1.1.7,"(21991, 21992, 21993)"
8216,21765,86456,6,290438,287464,S,,S1 - NTD,B.1.1.7,"(21765, 21766, 21767, 21768, 21769, 21770)"


In [13]:
ARG_deletion_events[ARG_deletion_events["node"] == 86456].to_latex()

'\\begin{tabular}{lrrrrrlllll}\n\\toprule\n & start & node & length & max_inheritors & min_inheritors & region & ORF1AB_region & spike_region & pango & deletion \\\\\n\\midrule\n8218 & 28271 & 86456 & 1 & 290446 & 290446 & non-coding &  &  & B.1.1.7 & (28271,) \\\\\n8215 & 11288 & 86456 & 9 & 290408 & 290402 & ORF1AB & NSP6 &  & B.1.1.7 & (11288, 11289, 11290, 11291, 11292, 11293, 11294, 11295, 11296) \\\\\n8217 & 21991 & 86456 & 3 & 288831 & 288192 & S &  & S1 - NTD & B.1.1.7 & (21991, 21992, 21993) \\\\\n8216 & 21765 & 86456 & 6 & 290438 & 287464 & S &  & S1 - NTD & B.1.1.7 & (21765, 21766, 21767, 21768, 21769, 21770) \\\\\n\\bottomrule\n\\end{tabular}\n'

Table 2: Deletions occuring on the node leading to the alpha variant. Two of these deletions are in the N-terminal domain of spike, with a further one in the non-coding region located between ORF8 and N, and another in NSP1. 

In [11]:
ARG_deletion_events[["start", "length", "region"]].value_counts().reset_index(name="number of occurrences")[:20]

,start,length,region,number of occurrences
0,29781,1,non-coding,12713
1,29762,1,non-coding,11356
2,29779,1,non-coding,10925
3,29769,1,non-coding,10783
4,29774,1,non-coding,10707
5,29700,1,non-coding,8543
6,29614,1,ORF10,5041
7,29555,1,non-coding,4816
8,29543,1,non-coding,4806
9,29541,1,non-coding,4804


In [14]:
ARG_deletion_events[["start", "length", "region"]].value_counts().reset_index(name="number of occurrences")[:20].to_latex()

'\\begin{tabular}{lrrlr}\n\\toprule\n & start & length & region & number of occurrences \\\\\n\\midrule\n0 & 29781 & 1 & non-coding & 12713 \\\\\n1 & 29762 & 1 & non-coding & 11356 \\\\\n2 & 29779 & 1 & non-coding & 10925 \\\\\n3 & 29769 & 1 & non-coding & 10783 \\\\\n4 & 29774 & 1 & non-coding & 10707 \\\\\n5 & 29700 & 1 & non-coding & 8543 \\\\\n6 & 29614 & 1 & ORF10 & 5041 \\\\\n7 & 29555 & 1 & non-coding & 4816 \\\\\n8 & 29543 & 1 & non-coding & 4806 \\\\\n9 & 29541 & 1 & non-coding & 4804 \\\\\n10 & 203 & 1 & non-coding & 3602 \\\\\n11 & 222 & 1 & non-coding & 3441 \\\\\n12 & 22013 & 6 & S & 2111 \\\\\n13 & 21991 & 3 & S & 2087 \\\\\n14 & 28254 & 1 & ORF8 & 1967 \\\\\n15 & 27556 & 64 & ORF7a & 1281 \\\\\n16 & 27579 & 3 & ORF7a & 1014 \\\\\n17 & 28093 & 1 & ORF8 & 898 \\\\\n18 & 21765 & 6 & S & 478 \\\\\n19 & 27532 & 1 & ORF7a & 403 \\\\\n\\bottomrule\n\\end{tabular}\n'

Table 3: The most recurrent deletions in the ARG. The majority of these highly recurent deletions are located in the non-coding region, but some are in coding regions such as 21991-21993 and 22013-22018 in the Spike protein. The 898 2bp deletion at position 21765 seems to be artifactual as it overwhelmingly occurs in the alpha variant, which has a longer 21765-21770 deletion as a defining deletion occuring in the node originating the lineage (node 86456; table 2).